In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

from FRVRS import nu, fu
from pandas import DataFrame
from itertools import product
from pysan.statistics import get_turbulence
from pysan import get_entropy, get_complexity

In [3]:

# load data frames
data_frames_list = nu.load_data_frames(frvrs_logs_df='frvrs_logs_df', distance_delta_df='distance_delta_df')
frvrs_logs_df = data_frames_list['frvrs_logs_df']
print(frvrs_logs_df.shape) # (829116, 125)
distance_delta_df = data_frames_list['distance_delta_df']
print(distance_delta_df.shape) # (873, 15)

Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/frvrs_logs_df.pkl.
Attempting to load /mnt/c/Users/DaveBabbitt/Documents/GitHub/itm-analysis-reporting/saves/pkl/distance_delta_df.pkl.
(829116, 125)
(873, 15)



# Create a DataFrame of all the Scene Stats

In [4]:

# Get the combo alphabet list
combos_alphabet = sorted(set([sequence_tuple[0] + '-' + sequence_tuple[1] for sequence_tuple in product(fu.patient_sort_order, fu.injury_severity_order)]))
combos_alphabet

['still-high', 'still-low', 'still-medium', 'walker-high', 'walker-low', 'walker-medium', 'waver-high', 'waver-low', 'waver-medium']

In [5]:

if nu.pickle_exists('scene_stats_df'): scene_stats_df = nu.load_object('scene_stats_df')
else:
    rows_list = []
    engagment_columns_list = ['patient_id', 'engagement_start', 'location_tuple', 'patient_sort', 'predicted_priority', 'injury_severity']
    for (session_uuid, scene_id), idx_df in distance_delta_df.groupby(fu.scene_groupby_columns):
        row_dict = list(idx_df.T.to_dict().values())[0]
        
        # Get the whole scene history
        mask_series = True
        for cn in fu.scene_groupby_columns: mask_series &= (frvrs_logs_df[cn] == eval(cn))
        scene_df = frvrs_logs_df[mask_series]
        
        # Get the engagement sequence and the stats from that
        actual_engagement_order = fu.get_engagement_starts_order(scene_df, verbose=False)
        if actual_engagement_order:
            df = DataFrame(actual_engagement_order, columns=engagment_columns_list)
            patient_sorts_list = df.apply(lambda row_series: row_series.patient_sort, axis='columns').tolist()
            patient_sorts_sequence, patient_sorts_string_to_integer_map = nu.convert_strings_to_integers(patient_sorts_list, alphabet_list=fu.patient_sort_order)
            injury_severitys_list = df.apply(lambda row_series: row_series.injury_severity, axis='columns').tolist()
            injury_severitys_sequence, injury_severitys_string_to_integer_map = nu.convert_strings_to_integers(injury_severitys_list, alphabet_list=fu.injury_severity_order)
            combos_list = [str(patient_sort) + '-' + str(injury_severity) for patient_sort, injury_severity in zip(patient_sorts_list, injury_severitys_list)]
            combos_sequence, combos_string_to_integer_map = nu.convert_strings_to_integers(combos_list, alphabet_list=combos_alphabet)
            
            # Replace negative values with 9s to maintain character width
            patient_sorts_sequence[patient_sorts_sequence < 0] = 9
            injury_severitys_sequence[injury_severitys_sequence < 0] = 9
            combos_sequence[combos_sequence < 0] = 9
            
            for fn in [get_turbulence, get_entropy, get_complexity]:
                function_name = fn.__name__
                for cn in ['patient_sort', 'injury_severity', 'combo']:
                    dictionary_key = function_name.replace('get', cn)
                    variable_name = f'{cn}s_sequence'
                    try:
                        dictionary_value = fn(list(eval(variable_name)))
                        row_dict[dictionary_key] = dictionary_value
                    except: continue
        
        row_dict['first_engagement'] = fu.get_first_engagement(scene_df)
        row_dict['first_treatment'] = fu.get_first_treatment(scene_df)
        row_dict['injury_correctly_treated_count'] = fu.get_injury_correctly_treated_count(scene_df)
        row_dict['injury_not_treated_count'] = fu.get_injury_not_treated_count(scene_df)
        row_dict['injury_treated_count'] = fu.get_injury_treated_count(scene_df)
        row_dict['injury_wrongly_treated_count'] = fu.get_injury_wrongly_treated_count(scene_df)
        row_dict['is_a_one_triage_file'] = fu.get_is_a_one_triage_file(scene_df)
        row_dict['is_scene_aborted'] = fu.get_is_scene_aborted(scene_df)
        row_dict['last_engagement'] = fu.get_last_engagement(scene_df)
        row_dict['logger_version'] = fu.get_logger_version(scene_df)
        row_dict['measure_of_right_ordering'] = fu.get_measure_of_right_ordering(scene_df)
        row_dict['patient_count'] = fu.get_patient_count(scene_df)
        row_dict['percent_hemorrhage_controlled'] = fu.get_percent_hemorrhage_controlled(scene_df)
        row_dict['pulse_taken_count'] = fu.get_pulse_taken_count(scene_df)
        row_dict['scene_end'] = fu.get_scene_end(scene_df)
        row_dict['scene_start'] = fu.get_scene_start(scene_df)
        row_dict['scene_type'] = fu.get_scene_type(scene_df)
        row_dict['stills_value'] = fu.get_stills_value(scene_df)
        row_dict['teleport_count'] = fu.get_teleport_count(scene_df)
        row_dict['time_to_last_hemorrhage_controlled'] = fu.get_time_to_last_hemorrhage_controlled(scene_df)
        row_dict['total_actions'] = fu.get_total_actions(scene_df)
        row_dict['triage_time'] = fu.get_triage_time(scene_df)
        row_dict['voice_capture_count'] = fu.get_voice_capture_count(scene_df)
        row_dict['walk_command_count'] = fu.get_walk_command_count(scene_df)
        row_dict['walk_value'] = fu.get_walk_value(scene_df)
        row_dict['walkers_value'] = fu.get_walkers_value(scene_df)
        row_dict['wave_command_count'] = fu.get_wave_command_count(scene_df)
        row_dict['wave_value'] = fu.get_wave_value(scene_df)
        rows_list.append(row_dict)
    scene_stats_df = DataFrame(rows_list)
    nu.store_objects(scene_stats_df=scene_stats_df)
    nu.save_data_frames(scene_stats_df=scene_stats_df)
print(scene_stats_df.shape) # (873, 50)

(873, 50)


In [6]:

scene_stats_df.sample(8).T

,8,472,31,505,48,328,559,356
session_uuid,040ba585-ccae-45ef-b4bd-a0e35ec222b0,868cbf3b-f04a-4e1a-a207-95881d529378,0667e56a-abe6-4097-9a56-33735f4855e2,8f6097ce-795d-483a-a811-e6a5423bc59d,0b41cfdc-0891-4d1d-91ea-ac86fa54a34c,5e1e1072-2244-4473-996d-e2cb337b805b,a3d6d913-7755-4e8d-a174-d5e491c4eac7,666ce61c-2ebc-40ee-902e-6f6aa42801ad
scene_id,0,0,0,1,1,1,1,0
last_still_engagement,51788.0,47873.0,354786.0,440277.0,96114.0,0.0,672223.0,NaN
actual_engagement_distance,11.110701,9.31076,31.826952,42.023434,43.587118,0.0,32.024568,0.0
ideal_engagement_distance,9.610701,9.31076,27.533722,41.940403,40.322467,0.0,58.06356,0.0
measure_of_ideal_ordering,0.998903,1.0,0.654365,0.453416,0.185552,NaN,-0.110314,NaN
distracted_engagement_distance,15.004686,13.857342,25.917138,31.024886,28.7644,0.0,32.402145,0.0
measure_of_distracted_ordering,-0.485968,-0.332604,0.781957,-0.063141,-0.004054,NaN,0.916583,NaN
measure_of_right_ordering,1.0,1.0,0.353345,0.820155,0.513449,NaN,-0.174099,NaN
actual_ideal_delta,1.5,0.0,4.29323,0.083031,3.264651,0.0,-26.038992,0.0
